<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Установка-необходимых-версий,-загрузки-библиотек-и-объявление-констант" data-toc-modified-id="Установка-необходимых-версий,-загрузки-библиотек-и-объявление-констант-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Установка необходимых версий, загрузки библиотек и объявление констант</a></span></li><li><span><a href="#Загрузка-данных-из-файла" data-toc-modified-id="Загрузка-данных-из-файла-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных из файла</a></span></li><li><span><a href="#Лемматизируем-столбец-text" data-toc-modified-id="Лемматизируем-столбец-text-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизируем столбец text</a></span></li><li><span><a href="#Очистим-данные-от-лишних-символов" data-toc-modified-id="Очистим-данные-от-лишних-символов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Очистим данные от лишних символов</a></span></li><li><span><a href="#Очистим-данные-от-лишних-слов" data-toc-modified-id="Очистим-данные-от-лишних-слов-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Очистим данные от лишних слов</a></span></li><li><span><a href="#Подсчитайте-величину-TF-IDF-для-текстов" data-toc-modified-id="Подсчитайте-величину-TF-IDF-для-текстов-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Подсчитайте величину TF-IDF для текстов</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация комментариев»

**Цель исследования:** Необходимо обучить модель классифицировать комментарии на позитивные и негативные. 

**Описание проекта:** Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Значение метрики качества *F1* должно быть не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv` с разметкой от токсичности правок. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Установка необходимых версий, загрузки библиотек и объявление констант

In [2]:
!pip install pymystem3

In [3]:
import pandas as pd
from pymystem3 import Mystem
import re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score
import time
from nltk.stem import WordNetLemmatizer
from tqdm.notebook import tqdm
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\peredkova\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


### Загрузка данных из файла

In [4]:
data = pd.read_csv('toxic_comments.csv', index_col=0)

In [5]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков нет, типы данных указаны верно.

### Лемматизируем столбец text

In [7]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(text): 
    list_word = nltk.word_tokenize(text)
    return ' '.join(wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word))

In [9]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Очистим данные от лишних символов

In [10]:
def clear_text(text):
    return " ". join(re.sub(r'[^a-zA-Z\']', ' ', text).split())

In [11]:
data['text'] = data['text'].apply(lambda x: clear_text(x)) 

In [12]:
data.head()

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D'aww He matches this background colour I'm se...,0
2,Hey man I'm really not trying to edit war It's...,0
3,More I can't make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0


### Очистим данные от лишних слов

In [13]:
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peredkova\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


### Подсчитайте величину TF-IDF для текстов

Посчитаем в пайплайне

Разделим данные на тренировочную и тестовую выборки, выделим на тест 10%, така как модель чувствительна к объему обучающих данных

In [14]:
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [15]:
X_train = train['text']
X_test = test['text']

In [16]:
y_train = train['toxic']
y_test = test['toxic']

## Обучение

Будем обучать две модели: LogisticRegression(), DecisionTreeClassifier()

**LogisticRegression()**

In [17]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stop_words)),
        ("model", LogisticRegression(solver='liblinear')),
    ]
)

С помощью TF-IDF мы получили очень длинные разряженные векторы. При использовании линейной модели регуляризации возрастает из-за большого количества признаков. Ее необходимо ослабить, подбирая гиперпараметр С.

In [18]:
params_logistic = {'model__C': [1, 5],
                  'vect__min_df': range(1, 3, 1)}

In [19]:
logistic_grid_search = GridSearchCV(estimator=pipeline, param_grid=params_logistic, scoring='f1', n_jobs=-1)

In [20]:
logistic_grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words=["hasn't",
                                                                    'herself',
                                                                    'so',
                                                                    'because',
                                                                    'again',
                                                                    'couldn',
                                                                    'into',
                                                                    'you',
                                                                    'our',
                                                                    'yourselves',
                                                                    'we',
                                                                    'doing',
                                                                    'above',
                                                                    'hers',
                                                                    'will',
                                                                    "couldn't",
                                                                    'but', 'by',
                                                                    'not', 'd',
                                                                    'did',
                                                                    'his',
                                                                    'until',
                                                                    "mustn't",
                                                                    "shouldn't",
                                                                    'too',
                                                                    'they', 't',
                                                                    'about',
                                                                    'such', ...])),
                                       ('model',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'model__C': [1, 5], 'vect__min_df': range(1, 3)},
             scoring='f1')

In [21]:
logistic_grid_search.best_params_

{'model__C': 5, 'vect__min_df': 2}

In [22]:
logistic_grid_search.best_score_

0.7687518672442702

In [23]:
logistic_grid_search.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(min_df=2,
                                 stop_words=["hasn't", 'herself', 'so',
                                             'because', 'again', 'couldn',
                                             'into', 'you', 'our', 'yourselves',
                                             'we', 'doing', 'above', 'hers',
                                             'will', "couldn't", 'but', 'by',
                                             'not', 'd', 'did', 'his', 'until',
                                             "mustn't", "shouldn't", 'too',
                                             'they', 't', 'about', 'such', ...])),
                ('model', LogisticRegression(C=5, solver='liblinear'))])

**DecisionTreeClassifier**

In [24]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stop_words)),
        ("model", DecisionTreeClassifier()),
    ]
)

In [25]:
params_tree = {
    'model__max_depth': range(1, 10),
    'model__min_samples_split': range(2, 10),
    'vect__min_df': range(1, 3, 1)
}

In [26]:
tree_grid_search = RandomizedSearchCV(estimator=pipeline, param_distributions=params_tree, scoring='f1', n_jobs=-1)

In [ ]:
tree_grid_search.fit(X_train, y_train)

In [ ]:
tree_grid_search.best_params_

In [ ]:
tree_grid_search.best_score_

In [ ]:
tree_grid_search.best_estimator_

Метрика у модели LogisticRegression на тренировочной выборке равна 0,76, это выше заявленного значения. Проведем тестирование этой модели

In [ ]:
y_pred = logistic_grid_search.predict(X_test)
tect_score = f1_score(y_pred, y_test)
tect_score

Оценка на тестовой выборке 0,78

## Выводы

В проекте я классифицировала комментарии на положительные и отрицательные. В начале я подготовила данные: лемматизировала столбец, очистила данные от лишних символов и слов, провела оценку важности слов.

Передо мной стояла задача бинарной классификации. Я построила две модели LogisticRegression и DecisionTreeClassifier и обучила их на тренировочный данных. У модели LogisticRegression получилась более высокая метрика качетсва 0,76. Я провели тестирование этой модели на тестовых данных и получила метрику 0,78, что соответствует заявленной в задании метрике качества.